# Build a RAG pipeline using ollama

Import all necessary libraries

In [59]:
import gradio as gr
import ollama
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

c:\Project_Files\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Define Web URL

In [ ]:
URL = ["https://ibighit.com/bts/eng/profile/","https://www.geeksforgeeks.org/"]

Define the loader

In [44]:
loader = WebBaseLoader(URL)

Load the web based URL

In [45]:
docs = loader.load()
docs

[Document(page_content='\n\n\n\n\n\n\n\n\nPROFILE | BTS | BIGHIT MUSIC\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nABOUT BTS\nBTS, an acronym of Bangtan Sonyeondan\r\n\t\t\t\t\t\t\tor “Beyond the Scene,” is a\r\n\t\t\t\t\t\tGrammy-nominated South Korean group that has been capturing the hearts of millions of fans\r\n\t\t\t\t\t\tglobally since its debut in June 2013. The members of BTS are RM, Jin, SUGA, j-hope, Jimin,\r\n\t\t\t\t\t\tV, and Jung Kook. Gaining recognition for their authentic and self-produced music, top-notch\r\n\t\t\t\t\t\tperformances, and the way they interact with their fans, BTS has established themselves as “21st century Pop Icons” breaking countless world\r\n\t\t\t\t\t\trecords. While imparting a positive influence through activities such as the LOVE MYSELF campaign and the UN ‘Speak Yourself’ speech, the band has mobilized millions\r\

Define the text splitter

In [46]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text_splitter

Split the documents into chunks

In [47]:
splits = text_splitter.split_documents(docs)
splits

[Document(page_content='PROFILE | BTS | BIGHIT MUSIC', metadata={'source': 'https://ibighit.com/bts/eng/profile/', 'title': 'PROFILE | BTS | BIGHIT MUSIC', 'description': 'Profile of BTS - RM, Jin, SUGA, j-hope, Jimin, V, Jung Kook', 'language': 'en'}),
 Document(page_content='ABOUT BTS\nBTS, an acronym of Bangtan Sonyeondan\r\n\t\t\t\t\t\t\tor “Beyond the Scene,” is a\r\n\t\t\t\t\t\tGrammy-nominated South Korean group that has been capturing the hearts of millions of fans\r\n\t\t\t\t\t\tglobally since its debut in June 2013. The members of BTS are RM, Jin, SUGA, j-hope, Jimin,\r\n\t\t\t\t\t\tV, and Jung Kook. Gaining recognition for their authentic and self-produced music, top-notch\r\n\t\t\t\t\t\tperformances, and the way they interact with their fans, BTS has established themselves as “21st century Pop Icons” breaking countless world\r\n\t\t\t\t\t\trecords. While imparting a positive influence through activities such as the LOVE MYSELF campaign and the UN ‘Speak Yourself’ speech, th

Create Ollama embeddings

In [48]:
embeddings = OllamaEmbeddings(model="mistral")

Dump the embedding into chroma vector store

In [51]:
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

Create the retriever

In [52]:
retriever = vectorstore.as_retriever()

Format the documents retrieved

In [53]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


Define the Ollama LLM function

In [54]:

def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response = ollama.chat(model='mistral', messages=[{'role': 'user', 'content': formatted_prompt}])
    return response['message']['content']


Define the RAG chain

In [55]:
def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = format_docs(retrieved_docs)
    return ollama_llm(question, formatted_context)


Use the RAG chain

In [56]:
result = rag_chain("What is URL is about?")
print(result)

 The context provided appears to be a list of various courses offered by an educational platform or institution. Topics covered range from Data Structures and Algorithms in different programming languages, System Design, DevOps Engineering, Software Testing & Automation, Full Stack Development, GATE Exam preparation among others. Interspersed within this list are brief profiles of individuals named Jin, Suga, j-hope, Jimin, V, and Jung Kook, with their respective dates of birth. The profiles mention BTS and BigHit Music, suggesting that these individuals might be members of the popular South Korean boy band BTS. Therefore, it's safe to infer that this document is about educational courses offered by an institution and brief biographies of some renowned figures associated with the music industry, possibly BTS.


RAG pipeline using ollama in gradio UI

In [62]:
# Function to load, split, and retrieve documents
def load_and_retrieve_docs(url):
    loader = WebBaseLoader(
        web_paths=(url,),
        bs_kwargs=dict() 
    )
    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    embeddings = OllamaEmbeddings(model="mistral")
    vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
    return vectorstore.as_retriever()

# Function to format documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Function that defines the RAG chain
def rag_chain(url, question):
    retriever = load_and_retrieve_docs(url)
    retrieved_docs = retriever.invoke(question)
    formatted_context = format_docs(retrieved_docs)
    formatted_prompt = f"Question: {question}\n\nContext: {formatted_context}"
    response = ollama.chat(model='mistral', messages=[{'role': 'user', 'content': formatted_prompt}])
    return response['message']['content']

# Gradio interface
iface = gr.Interface(
    fn=rag_chain,
    inputs=["text", "text"],
    outputs="text",
    title="RAG Chain Question Answering",
    description="Enter a URL and a query to get answers from the RAG chain."
)

# Launch the app
iface.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Reference:

https://www.youtube.com/watch?v=4HfSfFvLn9Q